## Additional Info about Pandas

## Must read

## Grouping with a custom aggregation function

pandas provides a number of aggregation functions to use with the groupby object. At some
point, you may need to write your own custom user-defined function that does not exist in
pandas or NumPy.

In this recipe, we use the college dataset to calculate the mean and standard deviation
of the undergraduate student population per state. We then use this information to find the
maximum number of standard deviations from the mean that any single population value
is per state.

Read in the college dataset, and find the mean and standard deviation of the
undergraduate population by state:

In [1]:
import pandas as pd
import numpy as np

In [31]:
college = pd.read_csv('data_1/college.csv')
(college
    .groupby('STABBR')
    ['UGDS']
    .agg(['mean', 'std'])
    .round(0)
)

,mean,std
STABBR,,
AK,2493.0,4052.0
AL,2790.0,4658.0
AR,1644.0,3143.0
AS,1276.0,NaN
AZ,4130.0,14894.0
...,...,...
VT,1513.0,2194.0
WA,2271.0,4124.0
WI,2655.0,4615.0


This output isn't quite what we desire. We are not looking for the mean and standard
deviations of the entire group but the maximum number of standard deviations away
from the mean for any one institution. To calculate this, we need to subtract the mean
undergraduate population by state from each institution's undergraduate population
and then divide by the standard deviation. This standardizes the undergraduate
population for each group. We can then take the maximum of the absolute value of
these scores to find the one that is farthest away from the mean. pandas does not
provide a function capable of doing this. Instead, we will need to create a custom
function:

In [79]:
def max_deviation(s):
    std_score = (s - s.mean()) / s.std()
    return std_score.abs().max()

After defining the function, pass it directly to the .agg method to complete the
aggregation. You will notice that the function name is placed inside the .agg method without
directly being called. Nowhere is the parameter s explicitly passed to max_deviation.
Instead, pandas implicitly passes the UGDS column as a Series to max_deviation.
The max_deviation function is called once for each group. As s is a Series, all normal
Series methods are available. It subtracts the mean of that particular grouping from each
of the values in the group before dividing by the standard deviation in a process called
standardization.

pandas defaults to using the sample standard deviation, which is undefined for any groups
with just a single value. For instance, the state abbreviation AS (American Samoa) has a
missing value returned as it has only a single institution in the dataset.

In [80]:
(college
    .groupby('STABBR')
    ['UGDS']
    .agg(max_deviation)
    .round(1)
)

STABBR
AK    2.6
AL    5.8
AR    6.3
AS    NaN
AZ    9.9
     ... 
VT    3.8
WA    6.6
WI    5.8
WV    7.2
WY    2.8
Name: UGDS, Length: 59, dtype: float64

You can also use your custom aggregation function along with the prebuilt functions. The
following does this and groups by state and religious affiliation:

In [81]:
(college
    .groupby(['STABBR', 'RELAFFIL']) 
    [['UGDS', 'SATVRMID', 'SATMTMID']] 
    .agg([max_deviation, 'mean', 'std'])
    .round(1)
)

UGDS                      SATVRMID               \
                max_deviation    mean     std max_deviation   mean   std   
STABBR RELAFFIL                                                            
AK     0                 2.1   3508.9  4539.5          NaN     NaN   NaN   
       1                 1.1    123.3   132.9          NaN   555.0   NaN   
AL     0                 5.2   3248.8  5102.4          1.6   514.9  56.5   
       1                 2.4    979.7   870.8          1.5   498.0  53.0   
AR     0                 5.8   1793.7  3401.6          1.9   481.1  37.9   
...                      ...      ...     ...          ...     ...   ...   
WI     0                 5.3   2879.1  5031.5          1.3   558.8  47.5   
       1                 3.4   1716.2  1934.6          2.1   500.1  66.0   
WV     0                 6.9   1873.9  6271.7          1.6   466.7  27.9   
       1                 1.3    716.4   503.6          1.9   485.7  14.6   
WY     0                 2.8   2244.4  2744.7          NaN   535.0   NaN   

                     SATMTMID               
                max_deviation   mean   std  
STABBR RELAFFIL                             
AK     0                 NaN     NaN   NaN  
       1                 NaN   503.0   NaN  
AL     0                 1.7   515.8  56.7  
       1                 1.4   485.6  61.4  
AR     0                 2.0   503.6  39.0  
...                      ...     ...   ...  
WI     0                 1.3   591.2  85.7  
       1                 1.8   526.6  42.5  
WV     0                 1.8   480.0  27.7  
       1                 1.7   484.8  17.7  
WY     0                 NaN   540.0   NaN  

[112 rows x 9 columns]

## Customizing aggregating functions with *args and **kwargs

When writing your own user-defined customized aggregation function, pandas implicitly
passes it each of the aggregating columns one at a time as a Series. Occasionally, you
will need to pass more arguments to your function than just the Series itself. To do so, you
need to be aware of Python's ability to pass an arbitrary number of arguments to functions.

The signature to .agg is __agg(func, *args, **kwargs)__. The func parameter is
a reducing function, the string name of a reducing method, a list of reducing functions,
or a dictionary mapping columns to functions or a list of functions. Additionally, as we have
seen, you can use keyword arguments to create named aggregations.

If you have a reducing function that takes additional arguments that you would like to use,
you can leverage the *args and **kwargs parameters to pass arguments to the reduction
function. You can use *args to pass an arbitrary number of positional arguments to your
customized aggregation function. Similarly, **kwargs allows you to pass an arbitrary
number of keyword arguments.

In this recipe, we will build a customized function for the college dataset that finds the
percentage of schools by state and religious affiliation that have an undergraduate population
between two values.

Define a function that returns the percentage of schools with an undergraduate
population of between 1,000 and 3,000:

In [33]:
def pct_between_1_3k(s):
    return (s
        .between(1_000, 3_000)
        .mean()
        * 100
    )

Calculate this percentage grouping by state and religious affiliation:

In [34]:
(college
    .groupby(['STABBR', 'RELAFFIL'])
    ['UGDS'] 
    .agg(pct_between_1_3k)
    .round(1)
)

STABBR  RELAFFIL
AK      0           14.3
        1            0.0
AL      0           23.6
        1           33.3
AR      0           27.9
                    ... 
WI      0           13.8
        1           36.0
WV      0           24.6
        1           37.5
WY      0           54.5
Name: UGDS, Length: 112, dtype: float64

This function works, but it does not give the user any flexibility to choose the lower
and upper bound. Let's create a new function that allows the user to parameterize
these bounds:

In [35]:
def pct_between(s, low, high):
    return s.between(low, high).mean() * 100

Pass this new function to the .agg method along with the lower and upper bounds:

In [39]:
(college
    .groupby(['STABBR', 'RELAFFIL'])
    ['UGDS'] 
    .agg(pct_between, 1_000, 10_000)
    .round(1)
)

STABBR  RELAFFIL
AK      0           42.9
        1            0.0
AL      0           45.8
        1           37.5
AR      0           39.7
                    ... 
WI      0           31.0
        1           44.0
WV      0           29.2
        1           37.5
WY      0           72.7
Name: UGDS, Length: 112, dtype: float64

There are a few ways we could achieve the same result in step 4. We could have explicitly
used keyword parameters to produce the same result:

In [37]:
(college
    .groupby(['STABBR', 'RELAFFIL'])
    ['UGDS'] 
    .agg(pct_between, high=10_000, low=1_000)
    .round(1)
)

STABBR  RELAFFIL
AK      0           42.9
        1            0.0
AL      0           45.8
        1           37.5
AR      0           39.7
                    ... 
WI      0           31.0
        1           44.0
WV      0           29.2
        1           37.5
WY      0           72.7
Name: UGDS, Length: 112, dtype: float64

## Examining the groupby object

The immediate result from using the .groupby method on a DataFrame is a groupby object.
Usually, we chain operations on this object to do aggregations or transformations without ever
storing the intermediate values in variables.

In this recipe, we examine the groupby object to examine individual groups.

Let's get started by grouping the state and religious affiliation columns from the
college dataset, saving the result to a variable and confirming its type:

In [83]:
college = pd.read_csv('data_1/college.csv')
grouped = college.groupby(['STABBR', 'RELAFFIL'])
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

Use the dir function to discover the attributes of a groupby object:

In [84]:
from pprint import pprint
pprint([attr for attr in dir(grouped) if not attr.startswith('_')])

['CITY',
 'CURROPER',
 'DISTANCEONLY',
 'GRAD_DEBT_MDN_SUPP',
 'HBCU',
 'INSTNM',
 'MD_EARN_WNE_P10',
 'MENONLY',
 'PCTFLOAN',
 'PCTPELL',
 'PPTUG_EF',
 'RELAFFIL',
 'SATMTMID',
 'SATVRMID',
 'STABBR',
 'UG25ABV',
 'UGDS',
 'UGDS_2MOR',
 'UGDS_AIAN',
 'UGDS_ASIAN',
 'UGDS_BLACK',
 'UGDS_HISP',
 'UGDS_NHPI',
 'UGDS_NRA',
 'UGDS_UNKN',
 'UGDS_WHITE',
 'WOMENONLY',
 'agg',
 'aggregate',
 'all',
 'any',
 'apply',
 'bfill',
 'boxplot',
 'corr',
 'corrwith',
 'count',
 'cov',
 'cumcount',
 'cummax',
 'cummin',
 'cumprod',
 'cumsum',
 'describe',
 'diff',
 'dtypes',
 'ewm',
 'expanding',
 'ffill',
 'fillna',
 'filter',
 'first',
 'get_group',
 'groups',
 'head',
 'hist',
 'idxmax',
 'idxmin',
 'indices',
 'last',
 'max',
 'mean',
 'median',
 'min',
 'ndim',
 'ngroup',
 'ngroups',
 'nth',
 'nunique',
 'ohlc',
 'pct_change',
 'pipe',
 'plot',
 'prod',
 'quantile',
 'rank',
 'resample',
 'rolling',
 'sample',
 'sem',
 'shift',
 'size',
 'skew',
 'std',
 'sum',
 'tail',
 'take',
 'transform',
 'v

We can find the number of groups with the `.ngroups` attribute:

In [45]:
grouped.ngroups

112

To find the uniquely identifying labels for each group, look in the .groups attribute,
which contains a dictionary of each unique group mapped to all the corresponding
index labels of that group. Because we grouped by two columns, each of the keys has
a tuple, one value for the STABBR column and another for the RELAFFIL column:

In [46]:
groups = list(grouped.groups)
groups[:6]

[('AK', 0), ('AK', 1), ('AL', 0), ('AL', 1), ('AR', 0), ('AR', 1)]

Retrieve a single group with the .get_group method by passing it a tuple of an
exact group label. For example, to get all the religiously affiliated schools in the state
of Florida, do the following:

In [85]:
grouped.get_group(('FL', 1))

,INSTNM,CITY,STABBR,HBCU,MENONLY,...,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
712,The Bapt...,Graceville,FL,0.0,0.0,...,0.5878,0.5602,0.3531,30800,20052
713,Barry Un...,Miami,FL,0.0,0.0,...,0.5045,0.6733,0.4361,44100,28250
714,Gooding ...,Panama City,FL,0.0,0.0,...,NaN,NaN,NaN,NaN,PrivacyS...
715,Bethune-...,Daytona ...,FL,1.0,0.0,...,0.7758,0.8867,0.0647,29400,36250
724,Johnson ...,Kissimmee,FL,0.0,0.0,...,0.6689,0.7384,0.2185,26300,20199
...,...,...,...,...,...,...,...,...,...,...,...
7486,Strayer ...,Coral Sp...,FL,NaN,NaN,...,NaN,NaN,NaN,49200,36173.5
7487,Strayer ...,Fort Lau...,FL,NaN,NaN,...,NaN,NaN,NaN,49200,36173.5
7488,Strayer ...,Miramar,FL,NaN,NaN,...,NaN,NaN,NaN,49200,36173.5
7489,Strayer ...,Miami,FL,NaN,NaN,...,NaN,NaN,NaN,49200,36173.5


You can also call the .head method on your groupby object to get the first rows of
each group together in a single DataFrame:

In [86]:
grouped.head(2)

,INSTNM,CITY,STABBR,HBCU,MENONLY,...,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama ...,Normal,AL,1.0,0.0,...,0.7356,0.8284,0.1049,30300,33888
1,Universi...,Birmingham,AL,0.0,0.0,...,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge ...,Montgomery,AL,0.0,0.0,...,0.6801,0.7795,0.8540,40100,23370
10,Birmingh...,Birmingham,AL,0.0,0.0,...,0.1920,0.4809,0.0152,44200,27000
43,Prince I...,Elmhurst,IL,0.0,0.0,...,0.7857,0.9375,0.6569,PrivacyS...,20992
...,...,...,...,...,...,...,...,...,...,...,...
5289,Pacific ...,Mangilao,GU,0.0,0.0,...,0.9730,0.0000,0.2533,PrivacyS...,PrivacyS...
6439,Touro Un...,Henderson,NV,0.0,0.0,...,0.0000,0.2000,0.4000,NaN,PrivacyS...
7352,Marinell...,Henderson,NV,NaN,NaN,...,NaN,NaN,NaN,21200,9796.5
7404,Universi...,St. Croix,VI,NaN,NaN,...,NaN,NaN,NaN,31800,15150


## Filtering for states with a minority majority

Previously, we examined using Boolean arrays to filter rows. In a similar fashion, when using
the .groupby method, we can filter out groups. The .filter method of the groupby object
accepts a function that must return either True or False to indicate whether a group is kept.

This .filter method applied after a call to the .groupby method is completely different to
the DataFrame .filter method covered in the Selecting columns with methods recipe from
Chapter 2, Essential DataFrame Operations.

One thing to be aware of is that when the .filter method is applied, the result does not use
the grouping columns as the index, but keeps the original index! The DataFrame .filter
method filters columns, not values.

In this recipe, we use the college dataset to find all the states that have more non-white
undergraduate students than white. This is a dataset from the US, where whites form the
majority and therefore, we are looking for states with a minority majority.

Read in the college dataset, group by state, and display the total number of groups.
This should equal the number of unique states retrieved from the .nunique Series
method:

In [88]:
college = pd.read_csv('data/college.csv', index_col='INSTNM')
grouped = college.groupby('STABBR')
grouped.ngroups

59

In [89]:
college['STABBR'].nunique() # verifying the same number

59

The grouped variable has a `.filter` method, which accepts a custom function
that determines whether a group is kept. The custom function accepts a DataFrame
of the current group and is required to return a Boolean. Let's define a function
that calculates the total percentage of minority students and returns True if this
percentage is greater than a user-defined threshold:

In [90]:
def check_minority(df, threshold):
    minority_pct = 1 - df['UGDS_WHITE']
    total_minority = (df['UGDS'] * minority_pct).sum()
    total_ugds = df['UGDS'].sum()
    total_minority_pct = total_minority / total_ugds
    return total_minority_pct > threshold

Use the .filter method passed with the check_minority function and a
threshold of 50% to find all states that have a minority majority:

In [91]:
college_filtered = grouped.filter(check_minority, threshold=.5)
college_filtered

,CITY,STABBR,HBCU,MENONLY,WOMENONLY,...,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
INSTNM,,,,,,,,,,,
Everest College-Phoenix,Phoenix,AZ,0.0,0.0,0.0,...,0.8291,0.7151,0.6700,28600,9500
Collins College,Phoenix,AZ,0.0,0.0,0.0,...,0.7205,0.8228,0.4764,25700,47000
Empire Beauty School-Paradise Valley,Phoenix,AZ,0.0,0.0,0.0,...,0.6349,0.5873,0.4651,17800,9588
Empire Beauty School-Tucson,Tucson,AZ,0.0,0.0,0.0,...,0.7962,0.6615,0.4229,18200,9833
Thunderbird School of Global Management,Glendale,AZ,0.0,0.0,0.0,...,0.0000,0.0000,0.0000,118900,PrivacyS...
...,...,...,...,...,...,...,...,...,...,...,...
WestMed College - Merced,Merced,CA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,15623.5
Vantage College,El Paso,TX,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9500
SAE Institute of Technology San Francisco,Emeryville,CA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9500


Just looking at the output may not be indicative of what happened. The DataFrame
starts with the state of Arizona (AZ) and not Alaska (AK), so we can visually confirm
that something changed. Let's compare the shape of this filtered DataFrame with the
original. Looking at the results, about 60% of the rows have been filtered, and only
20 states remain that have a minority majority:

In [67]:
college.shape

(7535, 26)

In [68]:
college_filtered.shape

(3028, 26)

In [69]:
college_filtered['STABBR'].nunique()

20

## Transforming through a weight loss bet

The `transform` method in pandas is designed for performing group-wise operations on a DataFrame, allowing users to apply a function to each group independently and obtain results aligned with the original DataFrame's structure. It is a powerful tool for creating new columns or modifying existing ones based on group-specific calculations.

In this recipe, we use simulated data from two individuals to track the percentage of weight loss over four months. The scenario in this recipe will track weight loss from two individuals throughout a four-month period and determine a winner. At the end of each month, a winner will be declared based on the individual who lost the highest percentage of body weight for that month. To track weight loss, we group our data by month and person, and then call the .transform method to find the percentage weight loss change for each week against the start of the month. 

We will use the .transform method in this recipe. This method returns a new object that
preserves the index of the original DataFrame but allows you to do calculations on groups
of the data.

Read in the raw weight_loss dataset, and examine the first month of data from the two people, Amy and Bob. There are a total of four weigh-ins per month:

In [50]:
weight_loss = pd.read_csv('data_1/weight_loss.csv')
weight_loss.query('Month == "Jan"')

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
1,Amy,Jan,Week 1,197
2,Bob,Jan,Week 2,288
3,Amy,Jan,Week 2,189
4,Bob,Jan,Week 3,283
5,Amy,Jan,Week 3,189
6,Bob,Jan,Week 4,283
7,Amy,Jan,Week 4,190


To determine the winner for each month, we only need to compare weight loss from
the first week to the last week of each month. But, if we wanted to have weekly
updates, we can also calculate weight loss from the current week to the first week
of each month. Let's create a function that is capable of providing weekly updates.
It will take a Series and return a Series of the same size:

In [92]:
def percent_loss(s):
    return ((s - s.iloc[0]) / s.iloc[0]) * 100

Let's test out this function for Bob during the month of January:

In [63]:
(weight_loss
    .query('Name=="Bob" and Month=="Jan"')
    ['Weight']
    .pipe(percent_loss)
)

0    0.000000
2   -1.030928
4   -2.749141
6   -2.749141
Name: Weight, dtype: float64

After the first week, Bob lost 1% of his body weight. He continued losing weight during
the second week but made no progress during the last week. We can apply this
function to every single combination of person and month to get the weight loss per
week in relation to the first week of the month. To do this, we need to group our data
by Name and Month, and then use the .transform method to apply this custom
function. The function we pass to .transform needs to maintain the index of the
group that is passed into it, so we can use percent_loss here:

In [64]:
(weight_loss
    .groupby(['Name', 'Month'])
    ['Weight'] 
    .transform(percent_loss)
)

0     0.000000
1     0.000000
2    -1.030928
3    -4.060914
4    -2.749141
        ...   
27   -3.529412
28   -3.065134
29   -3.529412
30   -4.214559
31   -5.294118
Name: Weight, Length: 32, dtype: float64

The .transform method takes a function that returns an object with the same
index (and the same number of rows) as was passed into it. Because it has the
same index, we can insert it as a column. The .transform method is useful for
summarizing information from the groups and then adding it back to the original
DataFrame. We will also filter down to two months of data for Bob:

In [93]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Name=="Bob" and Month in ["Jan", "Feb"]')
)

,Name,Month,Week,Weight,percent_loss
0,Bob,Jan,Week 1,291,0.0
2,Bob,Jan,Week 2,288,-1.0
4,Bob,Jan,Week 3,283,-2.7
6,Bob,Jan,Week 4,283,-2.7
8,Bob,Feb,Week 1,283,0.0
10,Bob,Feb,Week 2,275,-2.8
12,Bob,Feb,Week 3,268,-5.3
14,Bob,Feb,Week 4,268,-5.3


Notice that the percentage of weight loss resets after the new month. With this new
percent_loss column, we can manually determine a winner but let's see whether
we can find a way to do this automatically. As the only week that matters is the last
week, let's select week 4:

In [94]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
)

,Name,Month,Week,Weight,percent_loss
6,Bob,Jan,Week 4,283,-2.7
7,Amy,Jan,Week 4,190,-3.6
14,Bob,Feb,Week 4,268,-5.3
15,Amy,Feb,Week 4,173,-8.9
22,Bob,Mar,Week 4,261,-2.6
23,Amy,Mar,Week 4,170,-1.7
30,Bob,Apr,Week 4,250,-4.2
31,Amy,Apr,Week 4,161,-5.3


This narrows down the weeks but still doesn't automatically find out the winner of
each month. Let's reshape this data with the .pivot method so that Bob's and
Amy's percent weight loss is side by side for each month:

In [67]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
    .pivot(index='Month', columns='Name',
           values='percent_loss')
)

Name,Amy,Bob
Month,,
Apr,-5.3,-4.2
Feb,-8.9,-5.3
Jan,-3.6,-2.7
Mar,-1.7,-2.6


This output makes it clearer who has won each month, but we can still go a couple
of steps further. NumPy has a vectorized if then else function called where,
which can map a Series or array of Booleans to other values. Let's create a column,
winner, with the name of the winner:

In [68]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
    .pivot(index='Month', columns='Name',
           values='percent_loss')
    .assign(winner=lambda df_:
            np.where(df_.Amy < df_.Bob, 'Amy', 'Bob'))
)

Name,Amy,Bob,winner
Month,,,
Apr,-5.3,-4.2,Amy
Feb,-8.9,-5.3,Amy
Jan,-3.6,-2.7,Amy
Mar,-1.7,-2.6,Bob


In Jupyter, you can highlight the winning percentage for each month using the
.style attribute:

In [112]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
    .pivot(index='Month', columns='Name',
           values='percent_loss')
    .assign(winner=lambda df_:
            np.where(df_.Amy < df_.Bob, 'Amy', 'Bob'))
    .style.highlight_min(axis=1)
)

Name,Amy,Bob,winner
Month,,,
Apr,-5.3,-4.2,Amy
Feb,-8.9,-5.3,Amy
Jan,-3.6,-2.7,Amy
Mar,-1.7,-2.6,Bob


Use the .value_counts method to return the final score as the number of months won:

In [69]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
    .pivot(index='Month', columns='Name',
           values='percent_loss')
    .assign(winner=lambda df_:
            np.where(df_.Amy < df_.Bob, 'Amy', 'Bob'))
    .winner
    .value_counts()
)

Amy    3
Bob    1
Name: winner, dtype: int64

## Calculating weighted mean SAT scores per state with apply

The groupby object has four methods that accept a function (or functions) to perform a
calculation on each group. These four methods are .agg, .filter, .transform, and
.apply. Each of the first three of these methods has a very specific output that the function
must return. .agg must return a scalar value, .filter must return a Boolean, and
.transform must return a Series or DataFrame with the same length as the passed group.
The .apply method, however, may return a scalar value, a Series, or even a DataFrame
of any shape, therefore making it very flexible. It is also called only once per group (on a
DataFrame), while the .transform and .agg methods get called once for each aggregating
column (on a Series). The .apply method's ability to return a single object when operating on
multiple columns at the same time makes the calculation in this recipe possible.

In this recipe, we calculate the weighted average of both the math and verbal SAT scores
per state from the college dataset. We weight the scores by the population of undergraduate
students per school.

Read in the college dataset, and drop any rows that have missing values in the UGDS,
SATMTMID, or SATVRMID columns. We do not want any missing values for those
columns:

In [52]:
college = pd.read_csv('data_1/college.csv')
subset = ['UGDS', 'SATMTMID', 'SATVRMID']
college2 = college.dropna(subset=subset)
college.shape

(7535, 27)

In [53]:
college2.shape

(1184, 27)

The vast majority of institutions do not have data for our three required columns,
but this is still more than enough data to continue. Next, create a user-defined
function to calculate the weighted average of the SAT math scores:

In [55]:
def weighted_math_average(df):
    weighted_math = df['UGDS'] * df['SATMTMID']
    return int(weighted_math.sum() / df['UGDS'].sum())

Group by state and pass this function to the .apply method. Because each group
has multiple columns and we want to reduce those to a single value, we need to use
.apply. The weighted_math_average function will be called once for each group
(not on the individual columns in the group):

In [56]:
college2.groupby('STABBR').apply(weighted_math_average)

STABBR
AK    503
AL    536
AR    529
AZ    569
CA    564
     ... 
VT    566
WA    555
WI    593
WV    500
WY    540
Length: 53, dtype: int64

We successfully returned a scalar value for each group. Let's take a small detour and
see what the outcome would have been by passing the same function to the .agg
method (which calls the function for every column):

In [57]:
(college2
    .groupby('STABBR')
    .agg(weighted_math_average)
)

KeyError: 'UGDS'

The weighted_math_average function gets applied to each non-aggregating
column in the DataFrame. If you try and limit the columns to just SATMTMID, you
will get an error as you won't have access to UGDS. So, the best way to complete
operations that act on multiple columns is with .apply:

In [58]:
(college2
    .groupby('STABBR')
    ['SATMTMID'] 
    .agg(weighted_math_average)
)

KeyError: 'UGDS'

A nice feature of .apply is that you can create multiple new columns by returning
a Series. The index of this returned Series will be the new column names. Let's
modify our function to calculate the weighted and arithmetic average for both SAT
scores along with the count of the number of institutions from each group. We return
these five values in a Series:

In [95]:
def weighted_average(df):
   weight_m = df['UGDS'] * df['SATMTMID']
   weight_v = df['UGDS'] * df['SATVRMID']
   wm_avg = weight_m.sum() / df['UGDS'].sum()
   wv_avg = weight_v.sum() / df['UGDS'].sum()
   data = {'w_math_avg': wm_avg,
           'w_verbal_avg': wv_avg,
           'math_avg': df['SATMTMID'].mean(),
           'verbal_avg': df['SATVRMID'].mean(),
           'count': len(df)
   }
   return pd.Series(data)

(college2
    .groupby('STABBR')
    .apply(weighted_average)
    .astype(int)
)

,w_math_avg,w_verbal_avg,math_avg,verbal_avg,count
STABBR,,,,,
AK,503,555,503,555,1
AL,536,533,504,508,21
AR,529,504,515,491,16
AZ,569,557,536,538,6
CA,564,539,562,549,72
...,...,...,...,...,...
VT,566,564,526,527,8
WA,555,541,551,548,18
WI,593,556,545,516,14


In addition to finding just the arithmetic and weighted means, let's also find the geometric and
harmonic means of both SAT columns and return the results as a DataFrame with rows as the
name of the type of mean and columns as the SAT type. To ease the burden on us, we use the
NumPy function average to compute the weighted average and the SciPy functions gmean and
hmean for geometric and harmonic means:

In [60]:
from scipy.stats import gmean, hmean
def calculate_means(df):
    df_means = pd.DataFrame(index=['Arithmetic', 'Weighted',
                                   'Geometric', 'Harmonic'])
    cols = ['SATMTMID', 'SATVRMID']
    for col in cols:
        arithmetic = df[col].mean()
        weighted = np.average(df[col], weights=df['UGDS'])
        geometric = gmean(df[col])
        harmonic = hmean(df[col])
        df_means[col] = [arithmetic, weighted,
                         geometric, harmonic]
    df_means['count'] = len(df)
    return df_means.astype(int)

(college2
    .groupby('STABBR')
    .apply(calculate_means)
)

SATMTMID  SATVRMID  count
STABBR                                      
AK     Arithmetic       503       555      1
       Weighted         503       555      1
       Geometric        503       555      1
       Harmonic         503       555      1
AL     Arithmetic       504       508     21
...                     ...       ...    ...
WV     Harmonic         480       472     17
WY     Arithmetic       540       535      1
       Weighted         540       535      1
       Geometric        540       534      1
       Harmonic         540       535      1

[212 rows x 3 columns]

## Grouping by continuous variables

When grouping in pandas, you typically use columns with discrete repeating values. If there
are no repeated values, then grouping would be pointless as there would only be one row
per group. Continuous numeric columns typically have few repeated values and are generally
not used to form groups. However, if we can transform columns with continuous values into a
discrete column by placing each value in a bin, rounding them, or using some other mapping,
then grouping with them makes sense.
In this recipe, we explore the flights dataset to discover the distribution of airlines for different
travel distances. This allows us, for example, to find the airline that makes the most flights
between 500 and 1,000 miles. To accomplish this, we use the pandas cut function to
discretize the distance of each flight flown.

In [98]:
flights = pd.read_csv('data_1/flights.csv')
flights

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,...,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,...,590,1905,65.0,0,0
1,1,1,4,UA,DEN,...,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,...,641,1453,35.0,0,0
3,1,1,4,AA,DFW,...,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,...,1363,2225,39.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
58487,12,31,4,AA,SFO,...,1464,1045,-19.0,0,0
58488,12,31,4,F9,LAS,...,414,2050,4.0,0,0
58489,12,31,4,OO,SFO,...,262,1956,-5.0,0,0
58490,12,31,4,WN,MSP,...,907,855,34.0,0,0


If we want to find the distribution of airlines over a range of distances, we need to
place the values of the DIST column into discrete bins. Let's use the pandas cut
function to split the data into five bins:

In [99]:
bins = [-np.inf, 200, 500, 1000, 2000, np.inf]
cuts = pd.cut(flights['DIST'], bins=bins)
cuts

0        (500.0, ...
1        (1000.0,...
2        (500.0, ...
3        (1000.0,...
4        (1000.0,...
            ...     
58487    (1000.0,...
58488    (200.0, ...
58489    (200.0, ...
58490    (500.0, ...
58491    (500.0, ...
Name: DIST, Length: 58492, dtype: category
Categories (5, interval[float64, right]): [(-inf, 2... < (200.0, ... < (500.0, ... < (1000.0,... < (2000.0,...]

An ordered categorical Series is created. To help get an idea of what happened, let's
count the values of each category:

In [100]:
cuts.value_counts()

DIST
(500.0, 1000.0]     20659
(200.0, 500.0]      15874
(1000.0, 2000.0]    14186
(2000.0, inf]        4054
(-inf, 200.0]        3719
Name: count, dtype: int64

The cuts Series can now be used to form groups. pandas allows you to pass many
types into the .groupby method. Pass the cuts Series to the .groupby method
and then call the .value_counts method on the AIRLINE column to find the
distribution for each distance group. Notice that SkyWest (OO) makes up 33% of
flights of less than 200 miles but only 16% of those between 200 and 500 miles:

In [101]:
(flights
    .groupby(cuts)
    ['AIRLINE']
    .value_counts(normalize=True) 
    .round(3)
)

C:\Users\azatov\AppData\Local\Temp\ipykernel_4056\1298420373.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(cuts)


DIST           AIRLINE
(-inf, 200.0]  OO         0.326
               EV         0.289
               MQ         0.211
               DL         0.086
               AA         0.052
                          ...  
(2000.0, inf]  AS         0.012
               F9         0.004
               EV         0.000
               MQ         0.000
               OO         0.000
Name: proportion, Length: 70, dtype: float64

In [107]:
labels=['Under an Hour', '1 Hour', '1-2 Hours',
        '2-4 Hours', '4+ Hours']
cuts2 = pd.cut(flights['DIST'], bins=bins, labels=labels)
(flights
   .groupby(cuts2, observed=False)
   ['AIRLINE']
   .value_counts(normalize=True) 
   .round(3) 
   .unstack() 
)

AIRLINE,AA,AS,B6,DL,EV,...,OO,UA,US,VX,WN
DIST,,,,,,,,,,,
Under an Hour,0.052,0.000,0.000,0.086,0.289,...,0.326,0.027,0.000,0.000,0.009
1 Hour,0.071,0.001,0.007,0.189,0.156,...,0.159,0.062,0.016,0.028,0.194
1-2 Hours,0.144,0.023,0.003,0.206,0.101,...,0.106,0.131,0.025,0.004,0.138
2-4 Hours,0.264,0.016,0.003,0.165,0.016,...,0.046,0.199,0.040,0.012,0.160
4+ Hours,0.212,0.012,0.080,0.171,0.000,...,0.000,0.289,0.065,0.074,0.046


## Finding the longest streak of on-time flights

One of the most important metrics for airlines is their on-time flight performance. The
Federal Aviation Administration considers a flight delayed when it arrives at least 15 minutes
later than its scheduled arrival time. pandas includes methods to calculate the total and
percentage of on-time flights per airline. While these basic summary statistics are an
important metric, there are other non-trivial calculations that are interesting, such as finding
the length of consecutive on-time flights for each airline at each of its origin airports.

In this recipe, we find the longest consecutive streak of on-time flights for each airline at
each origin airport. This requires each value in a column to be aware of the value immediately
following it. We make clever use of the .diff and .cumsum methods to find streaks before
applying this methodology to each of the groups.

Before we get started with the flights dataset, let's practice counting streaks of ones
with a small sample Series:

In [118]:
s = pd.Series([0, 1, 1, 0, 1, 1, 1, 0])
s

0    0
1    1
2    1
3    0
4    1
5    1
6    1
7    0
dtype: int64

Our final representation of the streaks of ones will be a Series of the same length
as the original with an independent count beginning from one for each streak. To get
started, let's use the .cumsum method:

In [119]:
s1 = s.cumsum()
s1

0    0
1    1
2    2
3    2
4    3
5    4
6    5
7    5
dtype: int64

We have now accumulated all the ones going down the Series. Let's multiply this
Series by the original:

In [120]:
s.mul(s1)

0    0
1    1
2    2
3    0
4    3
5    4
6    5
7    0
dtype: int64

We have only non-zero values where we originally had ones. This result is fairly close
to what we desire. We just need to restart each streak at one instead of where the
cumulative sum left off. Let's chain the .diff method, which subtracts the previous
value from the current:

In [121]:
s.mul(s1).diff()

0    NaN
1    1.0
2    1.0
3   -2.0
4    3.0
5    1.0
6    1.0
7   -5.0
dtype: float64

A negative value represents the end of a streak. We need to propagate the negative
values down the Series and use them to subtract away the excess accumulation from
step 2. To do this, we will make all non-negative values missing with the .where
method:

In [122]:
(s
    .mul(s.cumsum())
    .diff()
    .where(lambda x: x < 0)
)

0    NaN
1    NaN
2    NaN
3   -2.0
4    NaN
5    NaN
6    NaN
7   -5.0
dtype: float64

We can now propagate these values down with the .ffill method:

In [123]:
(s
    .mul(s.cumsum())
    .diff()
    .where(lambda x: x < 0)
    .ffill()
)

0    NaN
1    NaN
2    NaN
3   -2.0
4   -2.0
5   -2.0
6   -2.0
7   -5.0
dtype: float64

Finally, we can add this Series back to the cumulative sum to clear out the excess accumulation:

In [124]:
(s
    .mul(s.cumsum())
    .diff()
    .where(lambda x: x < 0)
    .ffill()
    .add(s.cumsum(), fill_value=0)
)

0    0.0
1    1.0
2    2.0
3    0.0
4    1.0
5    2.0
6    3.0
7    0.0
dtype: float64

Now that we have a working consecutive streak finder, we can find the longest streak
per airline and origin airport. Let's read in the flights dataset and create a column
to represent on-time arrival:

In [125]:
flights = pd.read_csv('data_1/flights.csv')
(flights
    .assign(ON_TIME=flights['ARR_DELAY'].lt(15).astype(int))
    [['AIRLINE', 'ORG_AIR', 'ON_TIME']]
)

,AIRLINE,ORG_AIR,ON_TIME
0,WN,LAX,0
1,UA,DEN,1
2,MQ,DFW,0
3,AA,DFW,1
4,WN,LAX,0
...,...,...,...
58487,AA,SFO,1
58488,F9,LAS,1
58489,OO,SFO,1
58490,WN,MSP,0


Use our logic from the first seven steps to define a function that returns the maximum
streak of ones for a given Series:

In [126]:
def max_streak(s):
    s1 = s.cumsum()
    return (s
       .mul(s1)
       .diff()
       .where(lambda x: x < 0) 
       .ffill()
       .add(s1, fill_value=0)
       .max()
    )

Find the maximum streak of on-time arrivals per airline and origin airport along with
the total number of flights and the percentage of on-time arrivals. First, sort the day
of the year and the scheduled departure time:

In [127]:
(flights
    .assign(ON_TIME=flights['ARR_DELAY'].lt(15).astype(int))
    .sort_values(['MONTH', 'DAY', 'SCHED_DEP']) 
    .groupby(['AIRLINE', 'ORG_AIR'])
    ['ON_TIME'] 
    .agg(['mean', 'size', max_streak])
    .round(2)
)

mean  size  max_streak
AIRLINE ORG_AIR                        
AA      ATL      0.82   233        15.0
        DEN      0.74   219        17.0
        DFW      0.78  4006        64.0
        IAH      0.80   196        24.0
        LAS      0.79   374        29.0
...               ...   ...         ...
WN      LAS      0.77  2031        39.0
        LAX      0.70  1135        23.0
        MSP      0.84   237        32.0
        PHX      0.77  1724        33.0
        SFO      0.76   445        17.0

[114 rows x 3 columns]

Homework:

**Homework Assignment 1: Analyzing Sales Data**

You are given a dataset containing sales data for an e-commerce website. The dataset (`task\sales_data.csv`) has the following columns:

- `Date`: Date of the sale.
- `Product`: Name of the product sold.
- `Category`: Category to which the product belongs.
- `Quantity`: Number of units sold.
- `Price`: Price per unit.

**Tasks:**

1. Group the data by the `Category` column and calculate the following aggregate statistics for each category:
   - Total quantity sold.
   - Average price per unit.
   - Maximum quantity sold in a single transaction.
2. Identify the top-selling product in each category based on the total quantity sold.
3. Find the date on which the highest total sales (quantity * price) occurred.

**Homework Assignment 2: Examining Customer Orders**

You have a dataset (`task\customer_orders.csv`) containing information about customer orders. The dataset has the following columns:

- `OrderID`: Unique identifier for each order.
- `CustomerID`: Unique identifier for each customer.
- `Product`: Name of the product ordered.
- `Quantity`: Number of units ordered.
- `Price`: Price per unit.

**Tasks:**

1. Group the data by `CustomerID` and filter out customers who have made less than 20 orders.
2. Identify customers who have ordered products with an average price per unit greater than $120.
3. Find the total quantity and total price for each product ordered, and filter out products that have a total quantity less than 5 units.

**Homework Assignment 3: Population Salary Analysis**

1. "task\population.db" sqlite database has `population` table.
2. "task\population salary analysis.xlsx" file defines Salary Band categories. <br />
    Read the data from population table and calculate following measures:
    - Percentage of population for each salary category;
    - Average salary in each salary category;
    - Median salary in each salary category;
    - Number of population in each salary category;
3. Calculate the same measures in each State

Note: Use SQL only to select data from database. All the other calculations should be done in python.